In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline


In [2]:
au_telefons = pd.read_excel('АУ.xlsx')
au_telefons = au_telefons.fillna('')

In [3]:
dom, mobil, work = au_telefons['Телефон домашний'].tolist(), au_telefons['Телефон мобильный'].tolist(), au_telefons['Телефон рабочий'].tolist()

In [4]:
pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if len([x for x in j if x.isdigit()]) == 10:
            key[i] = '7' + ''.join([x for x in j if x.isdigit()])    
        if len([x for x in j if x.isdigit()]) == 11:
            temp_arr = ''.join([x for x in j if x.isdigit()])
            if temp_arr[0] == '8':
                key[i] = '7' + temp_arr[1:]
            elif temp_arr[0] == '7':    
                key[i] = temp_arr




for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            key[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
            key[i] = key[i].lstrip(' ')
            key[i] = key[i].rstrip(' ')
            if re.search(pattern2, key[i]):
                key[i] = re.sub(pattern2, '', key[i])
            key[i] = ''.join([x for x in key[i] if x not in '()-'])  

In [5]:
for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            if len(j) == 7:
                key[i] = '7812' + key[i]
            elif len(j) == 11 and j[0] == '8':
                key[i] = '7' + j[1:]
            elif len(j) == 10:
                key[i] = '7' + j

In [6]:
def concate_lists(*args) -> list:   # соединим все номера в одну строку чтобы проще было искать
    temp_lst = []
    for i in range(len(args[0])):
        temp_lst.append(' '.join([str(x[i]) for x in args]))
    return temp_lst
            
lst_search = concate_lists(dom, mobil, work)        

In [7]:
lst_search

[' 79313563775 79119334312',
 ' 79213158985 ',
 ' 79213606165 ',
 '79213158985 79213158985 79626861114',
 '79219553033 79214405144 ',
 ' 79219807837 79214202289',
 '79062538186 79062538186 ',
 '78122427036 79052107253 78122427036',
 ' 79643425045 ',
 '79995270660 79995270660 ',
 ' 79213976725 ',
 ' 79213158985 ',
 '79219960772 79215590960 ',
 '79213604783 79119337142 ',
 ' 79112305098 ',
 '79217430109 79217430109 79218450494',
 ' 79219974784 79219437751',
 '79219805643 79219805643 ',
 ' 79219415201 79117054152',
 '79112126648 79112126648 ',
 '79215848434 79215848434 ',
 ' 79214487765 79062733314',
 ' 79217430109 ',
 '79112493365 79215943001 79112493365',
 '79213158985 79213158985 79626861114',
 '79119164456 79112479005 ',
 ' 79117592037 ',
 '79062689798 79062689798 ',
 '79995270660 79995270660 ',
 ' 79313015226 ',
 '79216500673 79219461798 79216500673',
 ' 79219823924 79218835256',
 '79219413349 79219641484 79219413349',
 '79112126648 79046366984 79112126648',
 '79062538186 79062538186

In [8]:
quart = pd.read_excel('Номера.xlsx', dtype={'Номер абонента' : str})

In [9]:
new_pattern = r'^7\d{10}$'

In [10]:
quart['Номер абонента'] = quart['Номер абонента'].str.replace(r'^8812', '7812', regex=True)

quart[~quart['Номер абонента'].str.match(pat=new_pattern)]

,Номер абонента,Источник
3941,33970440793,2gis


In [11]:
quart = quart.drop(3941)

In [12]:
quart[~quart['Номер абонента'].str.match(pat=new_pattern)]

,Номер абонента,Источник


In [13]:
lst_with_coincidenc = []
for index, row in enumerate(lst_search):
    lst_with_coincidenc.append('')
    for i in quart['Номер абонента']:
        if str(i) in row:
            lst_with_coincidenc[index] = str(i)
            break
            
            
    

In [14]:
au_telefons['Совпадение'] = lst_with_coincidenc

In [15]:
au_telefons_final = au_telefons[au_telefons['Совпадение'] != '']

In [16]:
au_telefons_final = au_telefons_final.merge(quart, left_on='Совпадение', right_on='Номер абонента', how='left')

In [17]:
au_telefons_final = au_telefons_final.fillna('')

In [19]:
au_telefons_final['Старый/Новый'] = au_telefons_final['Дата приема'] == au_telefons_final['Минимальная дата приема Пациента (без удаленных)']
au_telefons_final['Старый/Новый'] = au_telefons_final['Старый/Новый'].replace({True:'Новый', False:'Старый'})
new_clients_lst = au_telefons_final[au_telefons_final['Старый/Новый'] == 'Новый']['Номер карты'].unique()
au_telefons_final.drop('Старый/Новый', axis=1, inplace=True)
au_telefons_final['Старый/Новый'] = au_telefons_final['Номер карты'].isin(new_clients_lst)
au_telefons_final['Старый/Новый'] = au_telefons_final['Старый/Новый'].replace({True:'Новый', False:'Старый'})


In [20]:
au_telefons_final.to_excel('Обработано.xlsx', index=False)

In [47]:
#au_telefons_final[au_telefons_final['Дата'] <= au_telefons_final['Дата приема']].to_excel('АУ_обработано.xlsx', index=False)

In [51]:
len([i for i in lst_with_coincidenc if i != '']), len(lst_with_coincidenc)

(4148, 100905)

In [56]:
s = pd.Series(lst_search, name='Номера вместе')

In [ ]:


s = pd.Series(['dsadas', 'dddd', 'ssss', 'ababab'])
lst = ['a', 'b', 'x']